In [29]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
import joblib

In [30]:
# Load the dataset
data = pd.read_csv('/bank-full.csv', sep=';')

In [31]:
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [32]:
# Data preprocessing: Encode categorical features
label_encoders = {}
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [33]:
# Define features (X) and target (y)
X = data.drop('y', axis=1)  # Features
y = data['y']               # Target variable

In [34]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
# Normalize numerical features after splitting
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Fit and transform the training data
X_test = scaler.transform(X_test)        # Transform the test data

In [42]:
# Train the SVM model
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)  # Fit the model on the training data
print(svm_model)

SVC(kernel='linear', random_state=42)


In [43]:
# Make predictions
y_pred = svm_model.predict(X_test)

In [44]:
# # Evaluate the model
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Model Accuracy: {accuracy:.2f}")
# print("\nClassification Report:\n", classification_report(y_test, y_pred))



# Evaluate the model
from sklearn.metrics import accuracy_score, classification_report

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# Print classification report
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Model Accuracy: 0.88

Classification Report:
               precision    recall  f1-score   support

           0       0.88      1.00      0.94     11966
           1       0.00      0.00      0.00      1598

    accuracy                           0.88     13564
   macro avg       0.44      0.50      0.47     13564
weighted avg       0.78      0.88      0.83     13564



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [73]:
# # # Save the model and encoders if needed
# joblib.dump(svm_model, 'svm_model.pkl')
# joblib.dump(scaler, 'scaler.pkl')

# Load the saved scaler and label encoders
scaler = joblib.load('scaler.pkl')
svm_model = joblib.load('svm_model.pkl')
label_encoders = {}


In [82]:
# List of categorical columns encoded during training
categorical_columns = ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']

In [83]:
# Assuming you have saved the label encoders as well
for col in categorical_columns[:-1]:  # Exclude 'y' since it's the target
    label_encoders[col] = joblib.load(f'label_encoder_{col}.pkl')

In [87]:
# Example new customer data (replace with actual data)
#new_data = [[35, "management", "married", 1500, "yes", "no", "yes", 5, "may", 250, 1, -1, 0, "unknown"]]

new_data = [[35, "management", "married", "primary", "yes", 1500, "yes", "no", "unknown", 5, "may", 250, 1, -1, 0, "unknown"]]

In [88]:
new_data_df = pd.DataFrame(new_data, columns=['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome'])

In [89]:
# Now, encode categorical columns using the saved label encoders
for col in categorical_columns[:-1]:  # Exclude 'y' (target column)
    new_data_df[col] = label_encoders[col].transform([new_data_df[col][0]])[0]

In [90]:
# Scale the new data using the saved scaler
new_data_scaled = scaler.transform(new_data_df)

In [91]:
# Make a prediction using the trained SVM model
prediction = svm_model.predict(new_data_scaled)

In [92]:
# Output the prediction
print(f"Prediction: {prediction[0]}")

Prediction: 0


In [98]:


if 'y' in categorical_columns:
    y_encoder = label_encoders['y']
    prediction_label = y_encoder.inverse_transform([prediction[0]])[0]
    print(f"Prediction : {prediction_label}")

    # Interpret the result:
    if prediction_label == 1:
        print("The customer will subscribe to a term deposit.")
    else:
        print("The customer will not subscribe to a term deposit.")

Prediction : no
The customer will not subscribe to a term deposit.
